In [2]:
from pydantic import BaseModel,Field
from chromadb import PersistentClient
from tqdm import tqdm
from litellm import completion
import numpy as np
from sklearn.manifold import TSNE
from dotenv import load_dotenv
from openai import OpenAI

In [1]:
print("J")

J


In [3]:
load_dotenv(override=True)
model = "gpt-4.1-nano"
db_name = "preprocessed_db"
collection_name = "docs"
embedding_model = "text-embedding-small"
avg_chunk_size = 500

In [4]:
openai = OpenAI()

class Result(BaseModel):
  page_content:str
  metadata:dict

In [5]:
class Chunk(BaseModel):
  headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
  summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
  original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

  def as_result(self,document):
    metadata = {"source":document['source'],"type":document["type"]}
    print(self)
    return Result(page_content=self.headline+"\n\n"+self.summary+"\n\n"+self.original_text,metadata=metadata)

class Chunks(BaseModel):
  chunks:list[Chunk]



In [54]:
from pathlib import Path
import glob
def fetch_documents():
  documents = []

  for folder in Path("./").iterdir():
    if folder.is_dir() and folder.name not in ["evaluation", "implementation", "vector_db"]:
      doc_type = folder.name
      for file in glob.glob(f"{folder}/*.md"):
        with open(file,'r',encoding='utf-8') as f:
          documents.append({"type":doc_type,"source":file,"text":f.read()})
  print(f"Loaded {len(documents)} documents")
  return documents
      
documents = fetch_documents()

Loaded 76 documents


## STEP-2 Make the Chunks

In [55]:
sum = []
def make_prompt(document):
  how_many = (len(document['text']) // 500) +1
  sum.append(how_many)
  return f"""
  You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
  """


In [53]:
print(make_prompt(documents[0]))

TypeError: string indices must be integers, not 'str'

In [56]:
def make_messages(document):
  return [{'role':'user','content':make_prompt(document)}]

In [57]:
def process_document(document):
  messsages = make_messages(document)
  response = completion(model=model,messages=messsages,response_format=Chunks)
  reply =response.choices[0].message.content
  doc_as_chunks = Chunks.model_validate_json(reply).chunks
  return [chunk.as_result(document) for chunk in doc_as_chunks]


In [11]:
process_document(documents[0])

headline='Introduction and Foundation' summary='Insurellm was founded in 2015 by Avery Lancaster as an innovative insurance tech startup, initially offering a marketplace connecting consumers with insurance providers.' original_text='Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.'
headline='Growth and Expansion (2015-2020)' summary='The company experienced rapid growth, expanding its product portfolio to include auto and home insurance portals and an enterprise reinsurance platform, reaching 200 employees across 12 offices by 2020.' original_text='The company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 

[Result(page_content='Introduction and Foundation\n\nInsurellm was founded in 2015 by Avery Lancaster as an innovative insurance tech startup, initially offering a marketplace connecting consumers with insurance providers.\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.', metadata={'source': 'company\\about.md', 'type': 'company'}),
 Result(page_content='Growth and Expansion (2015-2020)\n\nThe company experienced rapid growth, expanding its product portfolio to include auto and home insurance portals and an enterprise reinsurance platform, reaching 200 employees across 12 offices by 2020.\n\nThe company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platf

In [58]:
def create_chunks(documents):
  chunks =[]
  for doc in tqdm(documents):
    chunks.extend(process_document(doc))
  return chunks

In [60]:
chunks = create_chunks(documents)
print(len(chunks))

  0%|          | 0/76 [00:00<?, ?it/s]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



  0%|          | 0/76 [00:02<?, ?it/s]


InternalServerError: litellm.InternalServerError: InternalServerError: OpenAIException - Connection error.

In [ ]:
chunks[0]

Result(page_content='Introduction and Founding of Insurellm\n\nInsurellm was established in 2015 by Avery Lancaster as an innovative insurance tech startup aiming to transform the industry. Its initial product was Markellm, a marketplace linking consumers with insurance providers.\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.', metadata={'source': 'company\\about.md', 'type': 'company'})

## Save embeddings

In [17]:
def create_embeddings(chunks):
  chroma = PersistentClient(path=db_name)
  if collection_name in [c.name for c in chroma.list_collections()]:
    chroma.delete.collection(collection_name)

  texts = [chunk.page_content for chunk in chunks]
  emb = openai.embeddings.create(model="text-embedding-3-small",input=texts).data
  vectors = [e.embedding for e in emb]

  collection = chroma.get_or_create_collection(collection_name)

  ids = [str(i) for i in range(len(chunks))]
  metas = [chunk.metadata for chunk in chunks]
  collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
  print(f"Vectorstore created with {collection.count()} documents")

In [18]:
create_embeddings(chunks)

AttributeError: 'Client' object has no attribute 'delete'

In [26]:
chroma = PersistentClient(path=db_name)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings','documents','metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas=result['metadatas']
doc_type = [metadata['type'] for metadata in metadatas]
colors = [['blue','green','red','orange'][['products','employees','contracts','company'].index(t)] for t in doc_type]

tsne = TSNE(n_components=2,random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

import plotly.express as ex

hover_text =[
  f"Text:{doc[:20]}" for doc in (documents)
]

fig_2d = ex.scatter(
  x=reduced_vectors[:,0],
  y=reduced_vectors[:,1],
  color=doc_type,
  hover_data={
    "text":hover_text
  }
)

tsne = TSNE(n_components=3,random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

fig_3d = ex.scatter_3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    color=doc_type,
    hover_data={"text": hover_text}
)



In [27]:
fig_2d.show()

In [28]:
fig_3d.show()

In [29]:
class RankOrder(BaseModel):
  order:list[int] = Field(
    description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
  )

In [40]:
len(chunks[0].page_content)

496

In [51]:
def rerank(question, chunks):
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
"""
    user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
    user_prompt += "Here are the chunks \n\n"

    for index,chunk in enumerate(chunks):
        user_prompt += f"# Chunk ID: {index+1}:\n\n {chunk.page_content}"
    user_prompt+="Reply only with list od ranked ids,nothing else"
    response = completion(model=model,messages=messages,response_format=RankOrder)
    reply = response.choices[0].message.content
    order = RankOrder.model_validate_json(reply).order
    print(order)
    return [chunks[i-1] for i in order]

    
    

In [ ]:
RETERIEVAL_K = 10 
def fetch

1